In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder

In [ ]:
data_dir = "/kaggle/input/plantvillage-dataset/color"
image_size = 128
batch_size = 64
learn_rate = 0.01
epochs = 100

In [ ]:
def detect_plant_disease(image):
    
    
input_text = test_sentences[1]

for input_text in test_sentences:
    input = preprocess_text(input_text)

    score = text_model.predict(input)

    emotions = ['sadness (0)', 'joy (1)', 'love (2)', 'anger (3)', 'fear (4)', 'surprise (5)']

    predicted_class = np.argmax(score)
    certainty = 100 * np.max(score)

    print(f'Input Sentence: {input_text}')
    print(f'Predicted: {emotions[predicted_class]}')
    print(f'Certainty: {certainty:.2f}%')